# Persistence

We're going to discuss general IO, how hard disks work, and finally how to design a robust file system.

# Intro to I/O Devices

Computers are designed as a hierarchy of devices which will have different speeds.

CPU/Mem Bus is the fastest.

Below this are busses that connect to other devices:

- Graphics card
- Hard Disk
- Output device (monitors)
- Bluetooth card
- keyboard mouse, USB I/O
- RGB lights
- fans

Accessing any IO device (except maybe the graphics card) is atleast an order of magnitude slower than executing instructions on the CPU.

Many I/O devices have their own microprocessor of dedicated mini CPU so that they can execute independently from the main CPU.

This allows the OS to delegate the work of performing the actual IO to devices.

The OS makes requests of the hard disk.

These requests are made through outward facing registers and a memory buffer:
- COMMAND
- STATUS
- DATA

The OS likes to write the disk is 4KB chunks.

To write to some file, the OS will write the "write" command to COMMAND, then it will copy the 4KB to be written from memory to DATA.

Then the hard disk will write the data to the disk.

STATUS is used by the hard disk to post its status (busy, error statuses, etc).



**A generic Hard Disk Protocol**

```
while(STATUS == busy):
    spin;
write data -> DATA
write command -> COMMAND
while(STATUS == busy):
    spin;
```

This is horribly innefficient.

## Hard Disk Interrupts

To avoid the waiting and spinning, I/O devices use interrupts.

When a process makes an IO request, the OS will issue the command to the hard disk, then put the process to sleep in the BLOCKED state.

When the IO completes, the disk issues an interrupt, triggering the OS to run and change the state of the process from BLOCKED to READY.



## Optimizing writing the data

The OS likes to write in increments of 4KB.

When a process want to write 4KB to disk, the OS must must copy every byte (all 4096 of them) from memory to DATA.

During this time, the OS isn't running other processes, it is handling this request.

For efficiencies sake, this job is dedicated to a piece of dedicated hardware: the **DMA** which stands for Direct Memory Access.

When an IO request comes through, the OS tells the DMA what range of memory to write and the DMA copies that memory to DATA and issues the IO request to the hard disk.

# An intro to Hard Disks

A hard disk has a physical spinning disk with multiple platters to which data is written.

The platters are aluminum with magnetized top and bottom layers.

Magnetic patterns are recorded to store 1's and 0's.

Disks are organized into a set of sectors (512 bytes).

Disks have concentric tracks which contain these sectors.

Every magnetized layer has an arm and head which read/write data from/to the disk.

## Key Constraints

To read some sector from the disk, the arm first has to move to the correct track. This is called **seeking**. Then the sector must rotate to the head. Then finally it is read as it rotates under the head.

`T_io = seek_time + rotation_time + transfer_time`

<br>
<img src="images/01-disk.png" width="500">
<br>